In [1]:
from tensorflow import keras
import numpy as np

D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
import os
import cv2
from tqdm import tqdm

In [4]:
path="D://datasets//body_part";
X=[];y=[];
def onehot(i):
    if(i=='face'):
        return [0,0,1]
    elif(i=='hand'):
        return [0,1,0]
    elif(i=='leg'):
        return [1,0,0]
for i in tqdm(os.listdir(path)):
    pn=os.path.join(path,i);
    for j in os.listdir(pn):
        img=cv2.imread(os.path.join(pn,j),cv2.IMREAD_GRAYSCALE)
        X.append(np.reshape(cv2.resize(img,(128,128)),(128,128,1)))
        y.append(onehot(i));

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.78s/it]


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y)

In [6]:
X_train_arr=np.array(X_train)
X_val_arr=np.array(X_val)
y_train_arr=np.array(y_train)
y_val_arr=np.array(y_val)

In [7]:
print(X_train_arr.shape)

(247, 128, 128, 1)


In [18]:
#for i in range(len(X_val_arr)):
#   X_val_arr[i]/=255
model=Sequential()
model.add(Convolution2D(32, (5, 5), activation='relu',border_mode='same',input_shape=(128,128,1)))

D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", input_shape=(128, 128,..., padding="same")`
  after removing the cwd from sys.path.


In [19]:
model.add(Convolution2D(64, (5, 5), activation='relu',border_mode='same'))
model.add(Convolution2D(128, (5, 5), activation='relu',border_mode='same'))
model.add(Convolution2D(256, (5, 5), activation='relu',border_mode='same'))
model.add(Convolution2D(256, (5, 5), activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(128, (5, 5), activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, (5, 5), activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32, (5, 5), activation='relu',border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", padding="same")`
  """Entry point for launching an IPython kernel.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", padding="same")`
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="relu", padding="same")`
  after removing the cwd from sys.path.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activat

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 128, 128, 32)      832       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 128, 64)      51264     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 128, 128, 128)     204928    
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 128, 128, 256)     819456    
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 128, 128, 256)     1638656   
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 64, 64, 128)       819328    
__________

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
history=model.fit(x=X_train_arr,y=y_train_arr,epochs=10,validation_data=(X_val_arr,y_val_arr))

Train on 247 samples, validate on 83 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[32,256,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_2/Adam/gradients/conv2d_21/convolution_grad/Conv2DBackpropInput}} = Conv2DBackpropInput[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_2/Adam/gradients/conv2d_21/convolution_grad/ShapeN, conv2d_21/kernel/read, training_2/Adam/gradients/conv2d_21/Relu_grad/ReluGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
print(history.history)

In [39]:
sum(history.history['val_acc'])/20

0.32530120517834127